### Discussion

- Data taken from the pytorch machine translation tutorial
- Used batching in this code
- Used concatenation based attention in the paper Effective Approaches to Attention-based Neural Machine Translation
- The BLUE score without attention is 0.4. With attention, it improves to 0.7
- For some reason, beam search gives bad BLUE score. This could be a bug, I need to check

In [1]:
# Generic imports
from collections import Counter
from gensim.models import Word2Vec
from random import random
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from torch.autograd import Variable
import torch.optim as optim
from torch import nn
from torchvision import models, transforms
import io
import unicodedata
import unicodedata
import string
import re
import random
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from IPython.core.debugger import Tracer
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
import math, time

# project specific imports
import dataloader
from my_models import EmbeddingLayer
from my_models import Encoder
from my_models import Decoder
from my_models import AttentionDecoder
from common_functions import compute_bleu
from common_functions import cudify
from common_functions import printWords
from common_functions import extractSentences
from common_functions import compute_bleu
from common_functions import getBlueScores

# project initializations
torch.backends.cudnn.enabled = False
random.seed(32)
np.random.seed(32)
torch.manual_seed(7)
torch.cuda.manual_seed_all(7)
softmax = nn.Softmax()

In [4]:
# CUDA
params = {}
params['USE_CUDA'] = True
params['gpu'] = 0

#Load data
fra_eng = 'data_att/eng-fra.txt'
data = dataloader.DataLoader(fra_eng, params['USE_CUDA'])
input_lang = data.input_lang

target_lang = data.output_lang
pairs = data.pairs
print ('DONE: Data loading')
# display(len(input_lang.word2index))
# display(len(output_lang.word2index))
display(input_lang.name)
display(target_lang.name)
display(input_lang.n_words)
display(target_lang.n_words)
# display(len(pairs))




Reading lines...
Read 135842 sentence pairs
Trimmed to 10853 sentence pairs
Counting words...
Counted words:
fra 4490
eng 2926
DONE: Data loading


'fra'

'eng'

4490

2926

In [5]:
# Configuration
params['epochs'] = 40
params['learning_rate'] = 0.001
params['hidden_size'] = 256
params['teacher_forcing_ratio'] = 0.5
params['print_every'] = 20
params['batch_size'] = 200
params['input_vocab_size'] = input_lang.n_words
params['target_vocab_size'] = target_lang.n_words
params['embed_size'] = 300
params['rnn_type'] = 'LSTM'
params['num_layers'] = 1
params['beamLen'] = 10
params['beamSize'] = 5
params['max_seq_size'] = 11

# State of training
current_epoch = 0

# Load models
encoder = Encoder(params)
# decoder  = Decoder(params)
decoder  = AttentionDecoder(params)
embedding_weights_input = np.random.random((params['input_vocab_size'], params['embed_size']))
embedding_weights_input[0,:] = np.zeros((1, params['embed_size']))
# print (embedding_weights_input)
embedding_layer_input = EmbeddingLayer(embedding_weights_input)
embedding_weights_target = np.random.random((params['target_vocab_size'], params['embed_size']))
embedding_weights_target[0,:] = np.zeros((1, params['embed_size']))
# print (embedding_weights_target)
embedding_layer_target = EmbeddingLayer(embedding_weights_target)

# Criterion and optimizers
optimizer = optim.Adam([{'params': encoder.parameters()}, {'params': decoder.parameters()}, {'params': embedding_layer_input.parameters()}, {'params': embedding_layer_target.parameters()}],
                       lr=params['learning_rate'])
criterion = nn.CrossEntropyLoss()

#Load saved models to continue training
if current_epoch > 0:
    checkpoint = torch.load('outputs/saved_model_' + str(current_epoch-1))
    encoder.load_state_dict(checkpoint['Encoder'])
    decoder.load_state_dict(checkpoint['Decoder'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    
if params['USE_CUDA']:
    encoder, decoder, embedding_layer_input, embedding_layer_target = cudify((encoder, decoder, embedding_layer_input, embedding_layer_target), params['gpu'])


In [6]:
def supervisedTrain(batch, encoder, decoder, embedding_layer_input, embedding_layer_target, params, optimizer, criterion, onlyForward=False, teacherForcingRatio = 1.0):
#     onlyForward = False
    optimizer.zero_grad()
    total_batch_loss = 0.0
    token_count = 0.0
    input_batch = batch['input']
    input_batch_len = batch['input_len']
    target_batch = batch['target']
    target_batch_len = batch['target_len']
    batch_size = input_batch.size(0)
    max_seq_len = input_batch.size(1)
    volatile = False
    if onlyForward:
        volatile = True

    # Embedding layer
    input_batch_embedding = embedding_layer_input(input_batch)
    target_batch_embedding = embedding_layer_target(target_batch)


    #Encoder
    input_batch_len_sorted, input_perm_idx = input_batch_len.sort(0, descending=True)
    _, input_perm_idx_resort = input_perm_idx.sort(0, descending = False)

    input_batch_encoder_hidden = encoder.initHidden(batch_size = batch_size)
    input_batch_embedding = input_batch_embedding.index_select(0, input_perm_idx)
    # input_batch_embedding = input_batch_embedding.transpose(0,1) #seq first
    packed_input_batch_embedding = pack_padded_sequence(input_batch_embedding.transpose(0,1), input_batch_len_sorted.cpu().data.numpy())
    packed_input_batch_encoding, input_batch_encoder_hidden = encoder(packed_input_batch_embedding, input_batch_encoder_hidden)
    output_input_batch_encoding, _ = pad_packed_sequence(packed_input_batch_encoding)
    output_input_batch_encoding = output_input_batch_encoding.transpose(0,1) # make it batch first
    output_input_batch_encoding = output_input_batch_encoding.index_select(0, input_perm_idx_resort)

    if params['rnn_type'] == 'LSTM':
        input_batch_encoder_hidden = (input_batch_encoder_hidden[0].index_select(1, input_perm_idx_resort), 
                                     input_batch_encoder_hidden[1].index_select(1, input_perm_idx_resort))
    else:
        input_batch_encoder_hidden = input_batch_encoder_hidden.index_select(1, input_perm_idx_resort)

    input_batch_encoding = input_batch_encoder_hidden[0].squeeze(0)


    # Decoder
    flag = ''
    output_tokens = torch.empty(params['batch_size'],max_seq_len)
## If we do not need to use attention, use this fast code with TF
## This code, passes the whole tensor in LSTM and get all outputs in one go
## No need to have a loop over the max_seq_len
#     if np.random.uniform(0,1) <= teacherForcingRatio:
#         decoder_output, decoder_hidden = decoder(target_batch_embedding.transpose(0,1), input_batch_encoder_hidden, isPacked=False)
#         decoder_output = decoder_output.transpose(0,1) # make it batch first
#         softmaxed_probs = F.softmax(decoder_output, 2)
#         values, indices = torch.topk(softmaxed_probs, 1, dim=2, largest=True, sorted=False)
#         output_tokens = indices.squeeze()
#         flag = 'tf'
    if np.random.uniform(0,1) <= teacherForcingRatio:
        for idx in range(0,max_seq_len):
            decoder_output_temp, decoder_hidden, attn_weights_temp = decoder(decoder_input.transpose(0,1), decoder_hidden, output_input_batch_encoding)
            decoder_output[idx,:,:] = decoder_output_temp.squeeze(0)
            softmax_probs = decoder_output_temp.squeeze(0) # make it batch first
            softmax_probs = F.softmax(softmax_probs, 1)
            values, indices = torch.topk(softmax_probs, 1, dim=1, largest = True, sorted=False)
            decoder_input = target_batch_embedding[:,idx+1,:].unsqueeze(1)
            output_tokens[:,idx] = indices.squeeze()
        decoder_output = decoder_output.transpose(0,1) # make it batch x max_seq x vocab_size
        flag = 'tf'
    else:
        decoder_output = torch.empty(max_seq_len, params['batch_size'], params['target_vocab_size'])
        if params['USE_CUDA']:
            decoder_output = cudify((decoder_output,), params['gpu'])
            decoder_output = decoder_output[0]
        decoder_input = target_batch_embedding.narrow(1,0,1)
        decoder_hidden = input_batch_encoder_hidden
        for idx in range(0,max_seq_len):
            decoder_output_temp, decoder_hidden, attn_weights_temp = decoder(decoder_input.transpose(0,1), decoder_hidden, output_input_batch_encoding)
            decoder_output[idx,:,:] = decoder_output_temp.squeeze(0)
            softmax_probs = decoder_output_temp.squeeze(0) # make it batch first
            softmax_probs = F.softmax(softmax_probs, 1)
            values, indices = torch.topk(softmax_probs, 1, dim=1, largest = True, sorted=False)
            decoder_input = embedding_layer_target(indices)
            output_tokens[:,idx] = indices.squeeze()
        decoder_output = decoder_output.transpose(0,1) # make it batch x max_seq x vocab_size
        flag = 'notf'

    output_probabilities = decoder_output.contiguous().view(-1, decoder_output.size(2))
    target_batch_flattened = target_batch.view(-1)
    loss = criterion(output_probabilities, target_batch_flattened)
    mask = target_batch_flattened!=0
    mask = mask.float()
    loss_masked = torch.mul(loss, mask)
    loss_masked = loss_masked.sum()

    if not onlyForward:
        loss_masked.backward()
        optimizer.step()

    total_batch_loss += loss_masked.data.cpu().numpy()
    token_count += mask.sum().data.cpu().numpy()
    return total_batch_loss, token_count, output_tokens, flag




In [7]:
number_of_batches = math.floor(data.data_size/params['batch_size'])
# number_of_val_batches = math.floor(data.datasize['val']/params['batch_size'])
indexes = np.arange(data.data_size)
# val_indexes = np.arange(data.datasize['val'])

#print (number_of_batches, number_of_val_batches)
print (number_of_batches)

encoder.train()
decoder.train()
embedding_layer_input.train()
embedding_layer_target.train()
tfRatio = params['teacher_forcing_ratio']
tfRatio = 0

print('Start: Training')
for epoch in range(current_epoch, params['epochs']):
# for epoch in range(current_epoch, 20):
    np.random.shuffle(indexes)
    start = time.time()
    
    avg_loss_arr = []
    loss_arr = []
    
    avgvalloss = 0.0
    val_tokens = 0.0
    
    totalloss = 0.0
    totaltokens = 0.0
    for batch_num in range(number_of_batches):
        batch_indexes = indexes[batch_num*params['batch_size']:(batch_num+1)*params['batch_size']]
        batch = data.getBatch(batch_indexes)
        losstotal_batch, tokencount_batch, output_tokens, flag = supervisedTrain(batch, encoder, decoder, embedding_layer_input, embedding_layer_target, params, optimizer, criterion, onlyForward=False, teacherForcingRatio = tfRatio)
        totalloss += losstotal_batch
        totaltokens += tokencount_batch
#         pertoken_avgloss_batch.append(loss[0]/loss[1])
        #if lr > min_lr:
            #lr *= lr_decay
            #for param_group in optimizer.param_groups:
            #	param_group['lr'] = lr
        if batch_num%20 == 0:
            #print ("Done Batch:", batch_num, "\tAverage Loss Per Batch:", avgloss/(batch_num+1), "\t Current Batch Loss: ", loss, "\tlr:",lr)
            print ("  Batch:%3d"% batch_num, "\tTime:%3.2f" % (time.time() - start),"\tAverage Loss:%1.4f" % (totalloss/totaltokens), "\tCurrent Batch Loss:%1.4f" % (losstotal_batch/tokencount_batch))

#     for batch_val_num in range(number_of_val_batches):
#         batch_val_indexes = val_indexes[batch_val_num*params['batch_size']:(batch_val_num+1)*params['batch_size']]
#         batch_val = data.getQBatch(batch_val_indexes, 'val')
#         val_loss = supervisedTrain(batch, QEncoder, QDecoder, embedding_layer, sampler, params, optimizer, criterion, onlyForward=True)
#         val_tokens += val_loss[1]
#         avgvalloss += val_loss[0]
#     # avgvalloss = 0.0
#     # val_tokens = 1

#     print ("Epoch:",epoch, "\tTime:", time.time() - start, "\tAverage Loss Per Batch::", avgloss/tokens, "\tAverage Validation Loss:", avgvalloss/val_tokens)
    print ("\x1b[31m", "Epoch:%2d"%epoch, "\tTime:%3.2f"% (time.time() - start), "\tAverage Loss:%1.4f"% (totalloss/totaltokens), "\x1b[0m")
#     torch.save({'epoch': epoch ,'AEncoder': QEncoder.state_dict(),'ADecoder': QDecoder.state_dict(), 'embedding_layer':embedding_layer.state_dict(), 'optimizer':optimizer.state_dict()}, "outputs/supervised_QBot_NoAttention_2Layers_"+str(epoch))

#     loss_arr = np.array(loss_arr)
#     np.save(open('outputs/loss_QBot_NoAttention_2Layers_'+str(epoch), 'wb+'), loss_arr)


54
Start: Training
  Batch:  0 	Time:0.15 	Average Loss:8.0302 	Current Batch Loss:8.0302
  Batch: 20 	Time:1.63 	Average Loss:4.8129 	Current Batch Loss:3.1379
  Batch: 40 	Time:3.05 	Average Loss:3.8576 	Current Batch Loss:2.6656
 Epoch: 0 	Time:4.00 	Average Loss:3.5544 
  Batch:  0 	Time:0.07 	Average Loss:2.4856 	Current Batch Loss:2.4856
  Batch: 20 	Time:1.47 	Average Loss:2.4452 	Current Batch Loss:2.3729
  Batch: 40 	Time:2.88 	Average Loss:2.4144 	Current Batch Loss:2.3263
 Epoch: 1 	Time:3.80 	Average Loss:2.3992 
  Batch:  0 	Time:0.07 	Average Loss:2.3941 	Current Batch Loss:2.3941
  Batch: 20 	Time:1.51 	Average Loss:2.3200 	Current Batch Loss:2.2784
  Batch: 40 	Time:2.95 	Average Loss:2.2948 	Current Batch Loss:2.3579
 Epoch: 2 	Time:3.89 	Average Loss:2.2852 
  Batch:  0 	Time:0.07 	Average Loss:2.2083 	Current Batch Loss:2.2083
  Batch: 20 	Time:1.48 	Average Loss:2.1488 	Current Batch Loss:2.1071
  Batch: 40 	Time:2.91 	Average Loss:2.1307 	Current Batch Loss:2.0994


  Batch: 20 	Time:1.49 	Average Loss:0.5172 	Current Batch Loss:0.5256
  Batch: 40 	Time:2.90 	Average Loss:0.5341 	Current Batch Loss:0.5710
 Epoch:31 	Time:3.82 	Average Loss:0.5403 
  Batch:  0 	Time:0.07 	Average Loss:0.5134 	Current Batch Loss:0.5134
  Batch: 20 	Time:1.50 	Average Loss:0.5166 	Current Batch Loss:0.5001
  Batch: 40 	Time:2.90 	Average Loss:0.5173 	Current Batch Loss:0.5314
 Epoch:32 	Time:3.83 	Average Loss:0.5246 
  Batch:  0 	Time:0.08 	Average Loss:0.4668 	Current Batch Loss:0.4668
  Batch: 20 	Time:1.48 	Average Loss:0.4661 	Current Batch Loss:0.4763
  Batch: 40 	Time:2.88 	Average Loss:0.4700 	Current Batch Loss:0.4260
 Epoch:33 	Time:3.78 	Average Loss:0.4778 
  Batch:  0 	Time:0.07 	Average Loss:0.3896 	Current Batch Loss:0.3896
  Batch: 20 	Time:1.47 	Average Loss:0.4500 	Current Batch Loss:0.4346
  Batch: 40 	Time:2.86 	Average Loss:0.4583 	Current Batch Loss:0.4145
 Epoch:34 	Time:3.81 	Average Loss:0.4617 
  Batch:  0 	Time:0.07 	Average Loss:0.4102 	Cu

In [49]:
def inferBatch(batch, encoder, decoder, embedding_layer_input, embedding_layer_target, params, teacherForcing = False):
    input_batch = batch['input']
    input_batch_len = batch['input_len']
    target_batch = batch['target']
    target_batch_len = batch['target_len']
    batch_size = input_batch.size(0)
    max_seq_len = input_batch.size(1)

    # Embedding layer
    input_batch_embedding = embedding_layer_input(input_batch)
    target_batch_embedding = embedding_layer_target(target_batch)


    #Encoder
    input_batch_len_sorted, input_perm_idx = input_batch_len.sort(0, descending=True)
    _, input_perm_idx_resort = input_perm_idx.sort(0, descending = False)

    input_batch_encoder_hidden = encoder.initHidden(batch_size = batch_size)
    input_batch_embedding = input_batch_embedding.index_select(0, input_perm_idx)
    # input_batch_embedding = input_batch_embedding.transpose(0,1) #seq first
    packed_input_batch_embedding = pack_padded_sequence(input_batch_embedding.transpose(0,1), input_batch_len_sorted.cpu().data.numpy())
    packed_input_batch_encoding, input_batch_encoder_hidden = encoder(packed_input_batch_embedding, input_batch_encoder_hidden)
    output_input_batch_encoding, _ = pad_packed_sequence(packed_input_batch_encoding)
    output_input_batch_encoding = output_input_batch_encoding.transpose(0,1) # make it batch first
    output_input_batch_encoding = output_input_batch_encoding.index_select(0, input_perm_idx_resort)

    if params['rnn_type'] == 'LSTM':
        input_batch_encoder_hidden = (input_batch_encoder_hidden[0].index_select(1, input_perm_idx_resort), 
                                     input_batch_encoder_hidden[1].index_select(1, input_perm_idx_resort))
    else:
        input_batch_encoder_hidden = input_batch_encoder_hidden.index_select(1, input_perm_idx_resort)
    input_batch_encoding = input_batch_encoder_hidden[0].squeeze(0)

    output_tokens = torch.empty(batch_size,params['beamLen'])
    ## Decoder
    sos_tokens = target_batch_embedding.narrow(1,0,1)
    decoder_input = sos_tokens
    decoder_hidden = input_batch_encoder_hidden
    for idx in range(0,params['beamLen']):
        print(decoder_input.size())
        print(decoder_hidden[0].size())
        print(output_input_batch_encoding.size())
        decoder_output, decoder_hidden, _ = decoder(decoder_input.transpose(0,1), decoder_hidden, output_input_batch_encoding, isPacked=False)
        output_probabilities = decoder_output.squeeze(0) # make it batch first
        output_probabilities = F.softmax(output_probabilities, 1)
        values, indices = torch.topk(output_probabilities, 1, dim=1, largest = True, sorted=False)
        decoder_input = embedding_layer_target(indices)
        if teacherForcing: # this is cheating, just to test
            decoder_input = target_batch_embedding[:,idx+1,:].unsqueeze(1)
        output_tokens[:,idx] = indices.squeeze()
        
    return output_tokens

def beamSearch(input_tokens, encoder_batch_hidden, encoder_outputs, embedding_layer, decoder, params, word2ind, ind2word, vocab_size):
    batch_size = input_tokens.size(0)
    sequence_length = params['beamLen']
    beam_size = params['beamSize']
    input_tokens = input_tokens.narrow(1,0,1) # just pick SOS

    input_tokens_embedding = embedding_layer(input_tokens)
#     print(input_tokens_embedding.size())
#     print(encoder_batch_hidden[0].size())
#     print(encoder_outputs.size())
    
    output_probabilities, encoder_batch_hidden, _ = decoder(input_tokens_embedding.transpose(0,1), encoder_batch_hidden, encoder_outputs, isPacked=False)
    if params['rnn_type'] == 'LSTM':
        encoder_batch_hidden = (encoder_batch_hidden[0].view(-1,1,params['hidden_size']), encoder_batch_hidden[1].view(-1,1,params['hidden_size']))
        beam_hidden_state = (encoder_batch_hidden[0].repeat(1,beam_size,1).view(params['num_layers'], -1, params['hidden_size']), encoder_batch_hidden[1].repeat(1,beam_size,1).view(params['num_layers'], -1, params['hidden_size']))
    else:
        encoder_batch_hidden = encoder_batch_hidden.view(-1,1,params['hidden_size'])
        beam_hidden_state = encoder_batch_hidden.repeat(1,beam_size,1).view(params['num_layers'], -1, params['hidden_size'])
    
    encoder_outputs = encoder_outputs.repeat(beam_size,1,1) # added by simar (hacked)
    
    output_probabilities = F.log_softmax(output_probabilities, 2)
    values, indices = torch.topk(output_probabilities, beam_size, dim=2, largest=True, sorted=False)
    sequence_all = Variable(torch.zeros(batch_size, beam_size, sequence_length).long())
    sequence = Variable(torch.zeros(sequence_length, batch_size).long())
    sequence_probabilities = Variable(torch.zeros(sequence_length, batch_size, beam_size).float())
    EOS_TOKEN = word2ind['EOS']
    masked_vector = Variable(torch.zeros(1, vocab_size).float())
    masked_vector = masked_vector - 99999
    masked_vector[0,0] = 0
#     print(masked_vector.size())

    indexer = Variable(torch.arange(batch_size).long().unsqueeze(1).expand_as(indices.squeeze(0))*beam_size)
    masking_batch_num = Variable(torch.arange(batch_size*beam_size).long())
    if params['USE_CUDA']:
        sequence_all = sequence_all.cuda(params['gpu'])
        sequence = sequence.cuda(params['gpu'])
        sequence_probabilities = sequence_probabilities.cuda(params['gpu'])
        indexer = indexer.cuda(params['gpu'])
        masking_batch_num = masking_batch_num.cuda(params['gpu'])
        masked_vector = masked_vector.cuda(params['gpu'])

    sequence_all[:,:,0] = indices
    sequence_probabilities[0] = values
    beam_probability_sum = values.clone().squeeze(0).unsqueeze(2)
    for current_index in range(1, sequence_length):
#         print (current_index)
        #Select next words
        current_input_words = sequence_all[:,:,current_index - 1].clone().view(-1,1)
        mask = sequence_all == EOS_TOKEN
        mask = torch.max(mask, 2)[0].view(-1,)
        lengths = (sequence_all != 0).float()
        lengths = torch.sum(lengths, 2).unsqueeze(2)
        current_input_words_embeddings = embedding_layer(current_input_words)
        current_input_words_embeddings = current_input_words_embeddings.transpose(0,1)

        #Pass through Decoder
#         print(current_input_words_embeddings.size())
#         print(beam_hidden_state[0].size())
#         print(encoder_outputs.size())
        
        current_output_probabilities, beam_hidden_state, _ = decoder(current_input_words_embeddings, beam_hidden_state, encoder_outputs, isPacked=False)
        current_output_probabilities = F.log_softmax(current_output_probabilities, 2).view(-1, current_output_probabilities.size(2))

        #Masking EOS
        masked_indexes = masking_batch_num[mask]
#         print(current_output_probabilities)
        if masked_indexes.nelement() > 0:
            masking_vectors = masked_vector.repeat(masked_indexes.size(0), 1)
            current_output_probabilities.index_copy_(0, masked_indexes, masking_vectors)

        current_output_probabilities = current_output_probabilities.view(batch_size, beam_size, -1)
        # lengths = lengths.expand_as(current_output_probabilities)

        #Update Parameters for next iteration
        current_total_probabilities = beam_probability_sum.expand_as(current_output_probabilities) + current_output_probabilities
        # current_total_probabilities = current_total_probabilities/lengths
        current_values, current_indices = torch.topk(current_total_probabilities.view(batch_size,-1), beam_size, dim=1, largest=True, sorted=False)
        next_indices = current_indices/vocab_size
        next_words = current_indices%vocab_size

        next_indices_adjusted = next_indices + indexer
        next_indices_adjusted = next_indices_adjusted.view(-1,)

        sequence_all = torch.index_select(sequence_all.view(-1, sequence_length), 0, next_indices_adjusted).view(-1, beam_size, sequence_length)
        sequence_all[:,:,current_index] = next_words

        if params['rnn_type'] == 'LSTM':
            beam_hidden_state = (beam_hidden_state[0].transpose(0,1), beam_hidden_state[1].transpose(0,1))
            next_beam_hidden_state = (torch.index_select(beam_hidden_state[0], 0, next_indices_adjusted), torch.index_select(beam_hidden_state[1], 0, next_indices_adjusted))
            next_beam_hidden_state = (next_beam_hidden_state[0].transpose(0,1), next_beam_hidden_state[1].transpose(0,1))
        else:
            beam_hidden_state = beam_hidden_state.transpose(0,1)
            next_beam_hidden_state = torch.index_select(beam_hidden_state, 0, next_indices_adjusted)
            next_beam_hidden_state = next_beam_hidden_state.transpose(0,1)

        beam_probability_sum = current_values.unsqueeze(2)/lengths
        beam_hidden_state = next_beam_hidden_state

    return beam_probability_sum, sequence_all.data.cpu().numpy(), input_tokens.data.cpu().numpy()
#     test_index = 25
#     print (lengths)
#     print (beam_probability_sum[test_index])
#     tokens = sequence_all.data.cpu().numpy()
#     for tok in tokens[test_index]:
#         print (" ".join([ind2word[x] for x in tok if x != 0]))

#     tokens = input_tokens.transpose(0,1).data.cpu().numpy()
#     print(tokens)
#     for tok in tokens[test_index]:
#         print (" ".join([ind2word[x] for x in tok if x != 0]))
#     aa    


def beamSearchBatch(batch, encoder, decoder, embedding_layer_input, embedding_layer_target, params):
    input_batch = batch['input']
    input_batch_len = batch['input_len']
    target_batch = batch['target']
    target_batch_len = batch['target_len']
    batch_size = input_batch.size(0)
    max_seq_len = input_batch.size(1)

    # Embedding layer
    input_batch_embedding = embedding_layer_input(input_batch)
    target_batch_embedding = embedding_layer_target(target_batch)


    #Encoder
    input_batch_len_sorted, input_perm_idx = input_batch_len.sort(0, descending=True)
    _, input_perm_idx_resort = input_perm_idx.sort(0, descending = False)

    input_batch_encoder_hidden = encoder.initHidden(batch_size = batch_size)
    input_batch_embedding = input_batch_embedding.index_select(0, input_perm_idx)
    # input_batch_embedding = input_batch_embedding.transpose(0,1) #seq first
    packed_input_batch_embedding = pack_padded_sequence(input_batch_embedding.transpose(0,1), input_batch_len_sorted.cpu().data.numpy())
    packed_input_batch_encoding, input_batch_encoder_hidden = encoder(packed_input_batch_embedding, input_batch_encoder_hidden)
    output_input_batch_encoding, _ = pad_packed_sequence(packed_input_batch_encoding)
    output_input_batch_encoding = output_input_batch_encoding.transpose(0,1) # make it batch first
    output_input_batch_encoding = output_input_batch_encoding.index_select(0, input_perm_idx_resort)

    if params['rnn_type'] == 'LSTM':
        input_batch_encoder_hidden = (input_batch_encoder_hidden[0].index_select(1, input_perm_idx_resort), 
                                     input_batch_encoder_hidden[1].index_select(1, input_perm_idx_resort))
    else:
        input_batch_encoder_hidden = input_batch_encoder_hidden.index_select(1, input_perm_idx_resort)
    input_batch_encoding = input_batch_encoder_hidden[0].squeeze(0)

#     print (output_input_batch_encoding.size())
#     print (input_batch_encoder_hidden[0].size())
#     print(input_batch.size())
    beam_probability_sum, sequence_all, input_tokens = beamSearch(input_batch, input_batch_encoder_hidden, output_input_batch_encoding, embedding_layer_target, decoder, params, target_lang.word2index, target_lang.index2word, target_lang.n_words)
    return sequence_all

## With attention

In [10]:
groundtruth_tokens = torch.LongTensor(0, 11) 
output_tokens_infer = torch.LongTensor(0, 11)
for batch_num in range(number_of_batches):
    batch_indexes = indexes[batch_num*params['batch_size']:(batch_num+1)*params['batch_size']]
    batch = data.getBatch(batch_indexes)
    output_tokens = inferBatch(batch, encoder, decoder, embedding_layer_input, embedding_layer_target, params, teacherForcing = False)
#     printWords(output_tokens)
    output_tokens_infer = torch.cat([output_tokens_infer, output_tokens.type(torch.LongTensor)],0)
    groundtruth_tokens = torch.cat([groundtruth_tokens, batch['target'].cpu()], 0)
output_sentences = extractSentences(output_tokens_infer, target_lang.index2word)
groundtruth_sentences = extractSentences(groundtruth_tokens, target_lang.index2word)

In [11]:
np.mean(getBlueScores(groundtruth_sentences[0:-53], output_sentences[0:-53]))

/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoothi

0.710557643804564

In [12]:
output_sentences

['SOS you re nice . EOS',
 'SOS he is hopelessly in love . EOS',
 'SOS i m waiting my my friend . EOS',
 'SOS they re in the garden class class EOS',
 'SOS you re productive . EOS',
 'SOS he is ashamed of ask questions . EOS',
 'SOS i m glad to see you . EOS',
 'SOS i m not like that . EOS',
 'SOS you re very talented . EOS',
 'SOS we re not here to arrest you . EOS',
 'SOS he is studying history at the university . EOS',
 'SOS i m not interested in your opinion . EOS',
 'SOS i m afraid of having trouble . EOS',
 'SOS you re being of . . EOS',
 'SOS i m going to call a ambulance . EOS',
 'SOS you re a good liar . EOS',
 'SOS he is very fond of music . EOS',
 'SOS i m not a teacher . EOS',
 'SOS we re safe now . EOS',
 'SOS she s a particularily interesting person . EOS',
 'SOS we re very similar . EOS',
 'SOS i m tidy . EOS',
 'SOS she is not low spirits today . EOS',
 'SOS i m doing it in spite of you .',
 'SOS he s always trying in the future . EOS',
 'SOS i m going to build a fire .

In [13]:
groundtruth_sentences

['SOS you re nice . EOS',
 'SOS he s hopelessly in love . EOS',
 'SOS i m waiting for my friend . EOS',
 'SOS they are in the same class . EOS',
 'SOS you re productive . EOS',
 'SOS he is ashamed to ask questions . EOS',
 'SOS i m happy to see you . EOS',
 'SOS i m not like that . EOS',
 'SOS you re very talented . EOS',
 'SOS we are not here to arrest you . EOS',
 'SOS he is studying history at the university . EOS',
 'SOS i m not interested in your opinion . EOS',
 'SOS i am afraid of having trouble . EOS',
 'SOS you re being watched . EOS',
 'SOS i m going to call an ambulance . EOS',
 'SOS you re a good liar . EOS',
 'SOS he is very fond of music . EOS',
 'SOS i am not a teacher . EOS',
 'SOS we re safe now . EOS',
 'SOS she is a particularily interesting person . EOS',
 'SOS we are very similar . EOS',
 'SOS i m tidy . EOS',
 'SOS she is in low spirits today . EOS',
 'SOS i m doing it in spite of you . EOS',
 'SOS he is always looking to the future . EOS',
 'SOS i m going to buil

#### Beam search

In [51]:
output_tokens_beamsearch = torch.LongTensor(0, 10)
for batch_num in range(number_of_batches):
    batch_indexes = indexes[batch_num*params['batch_size']:(batch_num+1)*params['batch_size']]
    batch = data.getBatch(batch_indexes)
    output_tokens = beamSearchBatch(batch, encoder, decoder, embedding_layer_input, embedding_layer_target, params)
    top_output_tokens = output_tokens[:,0,:]
    top_output_tokens = torch.LongTensor(top_output_tokens)
    printWords(top_output_tokens)
    output_tokens_beamsearch = torch.cat([output_tokens_beamsearch, top_output_tokens],0)
output_sentences_beam = extractSentences(output_tokens_beamsearch, target_lang.index2word)

['SOS', 'you', 're', 'nice', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'in', 'at', 'love', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'at', 'my', 'my', 'job', '.', 'EOS', 'PAD']
['SOS', 'they', 're', 'in', 'the', 'chair', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'careful', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 'm', 'afraid', 'of', 'last', 'questions', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'here', 'to', 'to', 'you', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'not', 'not', 'concerned', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'a', 'talented', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'so', 'worried', 'short', 'about', 'you', '.', 'EOS']
['SOS', 'he', 'is', 'a', 'tv', 'tv', 'morning', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'still', 'a', 'in', 'your', 'opinion', '.', 'EOS']
['SOS', 'i', 'm', 'accustomed', 'of', 'a', 'hard', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'on', 'of', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'waiting', 'for',

['SOS', 'i', 'm', 'not', 'different', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'done', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 's', 'his', 'own', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'a', 'archeologist', 's', 'is', '.', 'EOS', 'PAD']
['SOS', 'we', 're', 'in', 'my', 'best', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'fat', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 're', 'not', 'happy', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'stubborn', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'going', 'to', 'bed', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 'spread', 'a', 'a', 'child', 'job', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'swiss', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'sick', 'of', 'lying', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'am', 'not', 'proud', 'of', 'this', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'sick', 'of', 'a', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'a', 'shy', '.', 'EOS', 'PAD', 

['SOS', 'they', 're', 'all', 'alike', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'of', 'of', 'anything', '.', 'EOS', 'PAD']
['SOS', 'he', 'is', 'a', 'fine', 'right', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'up', 'a', 'person', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'they', 're', 'not', 'looking', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'no', 'my', 'real', 'anymore', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'free', 'to', 'go', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'alone', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'a', 'little', 'shy', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'all', 'to', 'home', 'him', '.', 'EOS', 'PAD']
['SOS', 'we', 'm', 'all', 'crazy', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'eating', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'satisfied', 'with', 'being', 'here', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'psyched', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'by', 'for',

['SOS', 'he', 's', 'an', 'little', 'for', 'time', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'trying', 'to', 'clean', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 'is', 'young', 'angry', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'fool', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'being', 'to', 'to', 'you', 'the', 'thing', '.']
['SOS', 'i', 'am', 'afraid', 'of', 'his', 'own', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'not', 'a', 'real', 'teacher', '.', 'EOS', 'PAD']
['SOS', 'he', 'is', 'our', 'his', 'health', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 's', 'my', 'only', 'friend', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'a', 'good', 'busy', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'they', 're', 'almost', 'here', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'double', 'parked', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'looking', 'to', 'hear', 'that', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'in', 'first', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'all', 'talking',

['SOS', 'i', 's', 'my', 'friend', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'full', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'drunk', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'a', 'good', 'boy', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'in', 'to', 'bed', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'back', 'again', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'prepared', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 're', 'always', 'always', 'home', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'different', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 'are', 'important', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'looking', 'to', 'be', 'here', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'fine', 'right', 'right', 'aren', 't', 'EOS', 'PAD']
['SOS', 'i', 'm', 'disappointed', 'as', 'as', 'as', 'him', '.', 'EOS']
['SOS', 'you', 're', 'very', 'intelligent', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he'

...

['SOS', 'you', 're', 'normal', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'glad', 'you', 're', 'over', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'too', 'confused', 'to', 'do', 'my', 'time', '.']
['SOS', 'we', 're', 'weak', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'as', 'old', 'for', 'him', '.', 'EOS', 'PAD']
['SOS', 'she', 's', 'easygoing', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'naughty', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'familiar', 'to', 'you', 'to', 'be', 'marriage', '.']
['SOS', 'he', 's', 't', 'not', 'a', 'manager', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'out', 'of', 'vacation', 'vacation', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'so', 'proud', 'of', 'you', '.', 'EOS', 'PAD']
['SOS', 'we', 're', 'not', 'fat', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'eating', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'allergic', 'to', 'last', 'medicine', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'sorry', 'b

['SOS', 'i', 'm', 'getting', 'back', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'afraid', 'of', 'barking', 'dogs', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'sued', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'still', 'all', 'worried', 'fool', '.', 'EOS', 'PAD']
['SOS', 'i', 'am', 'on', 'a', 'tv', '.', '.', 'EOS', 'PAD']
['SOS', 'she', 's', 'became', 'careful', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'making', 'progress', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'going', 'to', 'lose', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'am', 'determined', 'to', 'hate', 'the', 'girl', '!', 'EOS']
['SOS', 'i', 'am', 'worried', 'on', 'the', 'new', 'own', '.', 'EOS']
['SOS', 'you', 're', 'going', 'to', 'lose', 'teeth', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'not', 'a', 'criminal', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'busy', 'people', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 'are', 'going', 'off', 'aren', 't', 'tom', '.', 'EOS']
['SOS', 'i',

['SOS', 'you', 're', 'very', 'intelligent', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'a', 'in', 'a', 'oxford', 'team', '.', 'EOS']
['SOS', 'he', 's', 'acting', 'on', 'your', 'own', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'seriously', 'of', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'used', 'to', 'leave', 'at', 'anymore', '.', 'EOS']
['SOS', 'we', 're', 'going', 'shopping', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 'is', 'an', 'in', 'a', 'teacher', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'more', 'than', 'she', 'is', '.', 'EOS', 'PAD']
['SOS', 'i', 'am', 'sure', 'that', 'you', 'want', 'to', 'come', '.']
['SOS', 'i', 'm', 'from', 'portugal', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'sorry', 'i', 'have', 'have', 'you', 'you', '.']
['SOS', 'you', 're', 'not', 'old', 'happy', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 's', 'over', 'of', 'suggestions', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'happy', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's'

['SOS', 'we', 're', 'very', 'curious', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'more', 'about', 'something', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'early', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'fearless', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 's', 'certainly', 'above', 'forty', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'tired', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'at', 'at', 'the', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'lying', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'not', 'safe', 'there', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 'is', 'very', 'interested', 'in', 'his', 'wall', '.', 'EOS']
['SOS', 'we', 're', 'back', 'dancing', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'still', 'done', 'yet', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'too', 'slow', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'famous', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', '

['SOS', 'i', 'am', 'surprised', 'you', 're', 'in', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'careful', 'a', 'aren', 't', 't', 'you', '?']
['SOS', 'he', 's', 'a', 'good', 'good', 'meal', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'a', 'to', 'everything', 'all', 'tom', '.', 'EOS']
['SOS', 'you', 're', 'such', 'stupid', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'glad', 'to', 'liked', 'it', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'being', 'careful', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'surprised', 'to', 'see', 'you', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'going', 'to', 'be', 'it', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'very', 'enough', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'in', 'the', 'shower', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'the', 'one', 'who', 'met', 'him', '.', 'EOS']
['SOS', 'you', 're', 'very', 'sophisticated', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'too', 'of', 'to', 'me', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'definitely',

In [52]:
np.mean(getBlueScores(groundtruth_sentences[0:-53], output_sentences_beam))

/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoothi

0.2647851608256208

## Without attention

In [15]:
groundtruth_tokens = torch.LongTensor(0, 11) 
output_tokens_infer = torch.LongTensor(0, 11)
for batch_num in range(number_of_batches):
    batch_indexes = indexes[batch_num*params['batch_size']:(batch_num+1)*params['batch_size']]
    batch = data.getBatch(batch_indexes)
    output_tokens = inferBatch(batch, encoder, decoder, embedding_layer_input, embedding_layer_target, params, teacherForcing = False)
    printWords(output_tokens)
    output_tokens_infer = torch.cat([output_tokens_infer, output_tokens.type(torch.LongTensor)],0)
    groundtruth_tokens = torch.cat([groundtruth_tokens, batch['target'].cpu()], 0)
output_sentences = extractSentences(output_tokens_infer, target_lang.index2word)
groundtruth_sentences = extractSentences(groundtruth_tokens, target_lang.index2word)

['SOS', 'i', 'm', 'drinking', '.', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'as', 'you', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 's', 'a', 'teacher', 'of', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'all', 'of', 'of', 'that', '.', 'EOS', 'PAD']
['SOS', 'she', 'is', 'teaching', 'french', 'french', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 'scolded', 'him', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'all', 'in', 'the', 'now', 'now', '.', '.']
['SOS', 'i', 'm', 'used', 'to', 'to', 'it', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 't', 'aren', 't', 'you', '?', 'EOS', 'PAD']
['SOS', 'you', 're', 'so', 'talented', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'friends', 'friends', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'trying', 'for', 'for', 'a', 'car', '.', 'EOS']
['SOS', 'you', 're', 'in', 'of', 'booze', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'getting', 'with', 'the', 'wraps', '.', 'EOS', 'PAD']
['SOS', 'we', 're', 'going', 'to', 'we

['SOS', 'you', 're', 'creative', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'doctors', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'looking', 'right', 'right', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'all', 'racists', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'still', 'here', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'ready', 'to', 'go', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'sure', 'to', 'see', 'seeing', 'to', 'leave', '.']
['SOS', 'i', 'm', 'greedy', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'smart', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'unconscious', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'all', 'against', 'me', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'scared', 'to', 'get', '.', '.', 'EOS']
['SOS', 'i', 'm', 'not', 'persuaded', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'busy', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm

['SOS', 'he', 'is', 'on', 'his', 'his', '.', '.', 'EOS', 'PAD']
['SOS', 'she', 'stopped', 'picking', 'daisies', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'going', 'to', 'believe', 'this', '.', 'EOS']
['SOS', 'you', 're', 'a', 'me', 'blush', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'they', 're', 'dangerous', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'a', 'my', 'my', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'looking', 'with', 'my', 'work', '.', 'EOS', 'PAD']
['SOS', 'we', 're', 'not', 'going', 'to', 'you', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'sorry', 'i', 'hurt', 'you', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'finicky', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'not', '.', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'a', 'a', 'student', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'from', 'england', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 'is', 'always', 'finding', 'fault', 'with', 'the', 'years', '.']
['SOS', 'you', 're', 'skinny

['SOS', 'i', 'm', 'not', 'a', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'exhausted', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 'is', 'able', 'to', 'speak', 'a', 'languages', '.', 'EOS']
['SOS', 'you', 're', 'very', 'sophisticated', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'sure', 'he', 'will', 'll', 'come', '.', 'EOS']
['SOS', 'i', 'm', 'tired', 'tired', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'resilient', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'always', 'anticipating', 'good', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'retiring', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'always', 'of', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'they', 'are', 'in', 'the', 'the', 'the', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'my', 'my', 'friend', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'just', 'at', 'in', 'at', 'all', '.', 'EOS']
['SOS', 'she', 's', 'not', 'the', 'the', '.', '.', 'EOS', 'PAD']
['SOS', 'you',

['SOS', 'you', 're', 'resilient', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'studying', 'basque', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'going', 'to', 'to', 'to', 'some', 'the', '.']
['SOS', 'you', 're', 'crafty', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'sophisticated', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'starved', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'just', 'trying', 'for', '.', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'intelligent', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'normal', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'all', 'friends', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'not', 'a', 'at', 'at', '.', '.', 'EOS']
['SOS', 'you', 're', 'not', 'a', 'doctor', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'not', 'friends', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 's', 'a', 'a', 'a', 'world', '.', 'EOS', 'PAD']
['SOS', 'you', 'are', 'twi

...

['SOS', 'you', 're', 'very', 'attractive', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'alone', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'sharp', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'very', 'proud', 'of', 'i', 'can', '.', 'EOS']
['SOS', 'we', 're', 'unprejudiced', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 'is', 'busy', 'to', 'him', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 'are', 'the', 'my', 'of', '.', '.', 'EOS', 'PAD']
['SOS', 'we', 're', 'having', 'a', 'blast', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'they', 're', 'back', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'not', 'here', 'here', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'they', 're', 'not', 'mine', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'fascinating', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'not', 'aren', 't', 'you', '?', 'EOS', 'PAD']
['SOS', 'you', 're', 'so', 'sweet', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 'is'

['SOS', 'i', 'm', 'not', 'really', 'to', 'to', 'ready', 'here', '.']
['SOS', 'i', 'm', 'really', 'with', 'my', 'my', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'dehydrated', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 's', 'too', 'old', 'to', 'be', 'a', 'friend', '.']
['SOS', 'i', 'm', 'glad', 'that', 'can', 'did', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'very', 'very', 'in', 'in', 'work', 'work', '.']
['SOS', 'we', 're', 'not', 'really', 'sure', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 'is', 'always', 'grumbling', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'a', 'a', 'a', 'a', 'child', '.', 'EOS']
['SOS', 'i', 'm', 'sure', 'of', 'the', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'busy', 'busy', 'busy', 'here', '.', 'EOS', 'PAD']
['SOS', 'he', 'is', 'a', '.', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 'shouldn', 't', 'to', 'here', 'here', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'a', 'good', 'boy', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 's', 'a', 'beginner', '.', 'EOS',

['SOS', 'you', 're', 'not', 'so', 'interesting', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'taking', 'a', 'a', 'a', 'days', 'days', '.']
['SOS', 'i', 'm', 'pressing', 'charges', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'ready', 'we', 'can', 'make', 'safety', '.', 'EOS']
['SOS', 'i', 'm', 'the', 'for', 'for', 'for', 'the', 'job', '.']
['SOS', 'i', 'm', 'ready', 'to', 'to', 'the', 'help', '.', 'EOS']
['SOS', 'you', 're', 'all', 'good', 'cook', 'you', '?', 'EOS', 'PAD']
['SOS', 'you', 're', 'too', 'old', 'me', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'all', 'insane', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'at', 'the', 'the', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 'is', 'very', 'very', 'strong', '.', 'EOS', 'EOS', 'PAD']
['SOS', 'you', 're', 'overreacting', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'used', 'to', 'the', 'alone', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'fascinated', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'n

['SOS', 'you', 're', 'resourceful', 'resourceful', 'EOS', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'all', 'all', 'agreement', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 's', 'the', 'oldest', 'the', 'the', 'family', '.', 'EOS']
['SOS', 'i', 'm', 'really', 'busy', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'brave', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'stalling', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'still', 'your', 'friend', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'going', 'to', 'to', 'the', 'the', '.', '.']
['SOS', 'i', 'm', 'dying', 'today', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 's', 'a', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'very', 'impressed', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'very', 'a', 'of', 'a', 'tonight', '.', 'EOS']
['SOS', 'he', 'is', 'very', 'dependent', 'of', 'his', 'parents', '.', 'EOS']
['SOS', 'he', 'is', 'a', 'man', 'man', '.', 'EOS', 'PAD', 'PAD']
['SOS', 

['SOS', 'i', 'm', 'very', 'glad', 'to', 'see', 'you', '.', 'EOS']
['SOS', 'i', 'm', 'not', 'afraid', 'of', 'that', '.', 'EOS', 'PAD']
['SOS', 'she', 's', 'the', 'the', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'going', 'anywhere', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'right', 'right', 'right', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'from', 'of', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'not', 'that', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'all', 'home', 'home', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'resigning', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'coming', 'for', 'you', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'glad', 'that', 'makes', 'you', 'happy', '.', 'EOS']
['SOS', 'we', 're', 'getting', '.', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'that', 'you', 'you', 'you', '.', 'EOS']
['SOS', 'he', 'is', 'in', 'in', 'in', 'the', 'the', 'field', '.']
['SOS', 'i', 'm', 'not', 'busy', 'busy', 

In [16]:
np.mean(getBlueScores(groundtruth_sentences[0:-53], output_sentences[0:-53]))

/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


0.4347825391157951

In [17]:
output_sentences

['SOS i m drinking . . EOS',
 'SOS i m not as you . EOS',
 'SOS he s a teacher of . EOS',
 'SOS we re all of of that . EOS',
 'SOS she is teaching french french . EOS',
 'SOS she scolded him . EOS',
 'SOS i m all in the now now . .',
 'SOS i m used to to it . EOS',
 'SOS you re t aren t you ? EOS',
 'SOS you re so talented . EOS',
 'SOS we re friends friends . EOS',
 'SOS i m trying for for a car . EOS',
 'SOS you re in of booze . EOS',
 'SOS we re getting with the wraps . EOS',
 'SOS we re going to we can can can .',
 'SOS i m not very busy busy . EOS',
 'SOS i m as as as you . EOS',
 'SOS he s my friend friend . EOS',
 'SOS you re both my friends friends . EOS',
 'SOS i m loyal . EOS',
 'SOS i m not a anymore . EOS',
 'SOS i m not sure sure . EOS',
 'SOS he is very likely to come . EOS',
 'SOS i m too busy . EOS',
 'SOS i m glad to finally meet you . EOS',
 'SOS she is a a a hat . EOS',
 'SOS he s playing tv . EOS',
 'SOS i m not going to it . EOS',
 'SOS he is afraid of making mista

In [18]:
groundtruth_sentences

['SOS i m losing . EOS',
 'SOS i m not like you . EOS',
 'SOS he is a tennis player . EOS',
 'SOS we re all out of booze . EOS',
 'SOS she s teaching us french . EOS',
 'SOS she scolded him . EOS',
 'SOS i m now officially part of this group . EOS',
 'SOS i am forced to do it . EOS',
 'SOS you re afraid aren t you ? EOS',
 'SOS you re so talented . EOS',
 'SOS we re close friends . EOS',
 'SOS i m saving money for a car . EOS',
 'SOS you re out of booze . EOS',
 'SOS we are to eat at six . EOS',
 'SOS we re going to do everything we can . EOS',
 'SOS i m just not very busy . EOS',
 'SOS i m as old as you . EOS',
 'SOS he s my best friend . EOS',
 'SOS you re my friends both of you . EOS',
 'SOS i m loyal . EOS',
 'SOS i m not good looking . EOS',
 'SOS i m just not sure . EOS',
 'SOS he s very likely to come . EOS',
 'SOS i m too busy . EOS',
 'SOS i m glad to finally meet you . EOS',
 'SOS she s wearing a cool hat . EOS',
 'SOS he is playing golf . EOS',
 'SOS i m not going to sell it

#### Beam search

In [25]:
groundtruth_tokens = torch.LongTensor(0, 11) 

output_tokens_beamsearch = torch.LongTensor(0, 10)
for batch_num in range(number_of_batches):
    batch_indexes = indexes[batch_num*params['batch_size']:(batch_num+1)*params['batch_size']]
    batch = data.getBatch(batch_indexes)
    output_tokens = beamSearchBatch(batch, encoder, decoder, embedding_layer_input, embedding_layer_target, params)
    top_output_tokens = output_tokens[:,0,:]
    top_output_tokens = torch.LongTensor(top_output_tokens)
    printWords(top_output_tokens)
    output_tokens_beamsearch = torch.cat([output_tokens_beamsearch, top_output_tokens],0)
    groundtruth_tokens = torch.cat([groundtruth_tokens, batch['target'].cpu()], 0)
groundtruth_sentences = extractSentences(groundtruth_tokens, target_lang.index2word)    
output_sentences_beam = extractSentences(output_tokens_beamsearch, target_lang.index2word)

['SOS', 'i', 'm', 'untalented', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'as', 'you', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 's', 'a', 'student', 'of', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 'we', 're', 'all', 'of', 'of', '.', 'EOS', 'PAD']
['SOS', 'she', 's', 'teaching', 'french', 'french', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 'stood', 'him', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'all', 'now', 'now', 'now', 'now', '.', 'EOS']
['SOS', 'i', 'm', 'ready', 'to', 'to', 'it', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 't', 'aren', 't', 'you', '?', 'EOS', 'PAD']
['SOS', 'you', 're', 'so', 'talented', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'all', 'friends', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'trying', 'for', 'for', 'a', 'car', '.', 'EOS']
['SOS', 'you', 're', 'in', 'of', 'booze', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'getting', 'for', 'the', 'wraps', '.', 'EOS', 'PAD']
['SOS', 'we', 're', 'just', 'we', 'we', 

['SOS', 'i', 'm', 'not', '.', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'your', 'your', 'anymore', '.', 'EOS', 'PAD']
['SOS', 'we', 're', 'all', 'prisoners', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 'is', 'busy', 'and', 'her', '.', '.', 'EOS', 'PAD']
['SOS', 'she', 're', 'very', 'sophisticated', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'happy', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'sick', 'with', 'the', 'you', '.', 'EOS', 'PAD']
['SOS', 'we', 're', 'not', 'ready', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'happy', 'happy', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'naive', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 'stared', 'at', 'the', 'the', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'busy', 'good', 'good', 'good', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'not', 'very', 'busy', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'the', 'teacher', 'my', 'teacher', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 't

['SOS', 'i', 'm', 'ashamed', 'of', 'earthquakes', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'not', 'aren', 't', 'you', '?', 'EOS', 'PAD']
['SOS', 'i', 'm', 'glad', 'to', 'suggestions', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'broke', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'in', 'the', 'the', 'boat', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'crafty', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'going', 'to', 'go', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'a', 'a', 'a', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 's', 'a', 'doctor', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'bored', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'sick', 'of', 'of', 'this', 'weather', 'weather', '.']
['SOS', 'i', 'am', 'not', 'used', 'to', 'be', 'alone', '.', 'EOS']
['SOS', 'you', 're', 'a', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'astute', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'tired', 'tired', 'of'

['SOS', 'i', 'm', 'involved', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'naive', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'in', 'danger', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'beginning', 'to', 'be', 'curious', '.', 'EOS', 'PAD']
['SOS', 'they', 're', 'rich', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'sorry', 'i', 'yelled', 'at', 'you', '.', 'EOS']
['SOS', 'you', 're', 'very', 'emotional', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'not', 'happy', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'just', 'for', 'for', 'a', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'really', 'really', 'minded', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'not', 'my', 'friend', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 's', 'still', 'a', 'bad', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'still', '.', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'not', 'like', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'gl

['SOS', 'you', 're', 'reliable', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'delighted', 'to', 'have', 'a', 'a', '.', 'EOS']
['SOS', 'i', 'm', 'not', 'not', 'either', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'taller', 'than', 'me', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 'aren', 't', 't', 'are', 'you', '?', 'EOS', 'PAD']
['SOS', 'i', 'm', 'sure', 'i', 'half', 'awake', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'a', 'funny', 'guy', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'good', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 's', 'very', 'careful', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'going', 'to', 'figure', 'this', 'out', '.', 'EOS']
['SOS', 'we', 're', 'all', 'in', 'the', 'same', 'team', '.', 'EOS']
['SOS', 'he', 's', 'a', 'of', 'english', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'a', 'funny', 'guy', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'funny', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'not', 'very', 

...

['SOS', 'i', 'm', 'going', 'to', 'see', 'you', '.', '.', 'EOS']
['SOS', 'you', 're', 'old', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 's', 'sharp', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'as', 'smart', 'as', 'tom', '.', 'EOS']
['SOS', 'he', 's', 'a', 'student', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'free', 'to', 'the', 'tennis', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'skinny', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 's', 'innocent', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'still', 'right', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'here', 'here', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'in', 'in', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'very', 'hungry', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'in', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'sure', 'to', 'be', 'him', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'in', 'his', 'his', 'his', '.', 'EOS

['SOS', 'we', 're', 'both', 'teachers', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'am', 'in', 'with', 'my', 'friends', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'drunk', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'joking', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'just', 'my', 'my', 'mine', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'very', 'rude', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'she', 's', 'him', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'talented', 'talented', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'drunk', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'he', 'is', 'out', 'the', 'the', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 's', 'a', 'a', 'of', 'gentleman', '.', 'EOS', 'PAD']
['SOS', 'she', 's', 'she', 'she', 'feels', 'nervous', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'waiting', 'for', 'you', 'at', 'home', '.', 'EOS']
['SOS', 'i', 'm', 'very', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'looking

['SOS', 'you', 're', 'not', 'so', 'interesting', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'taking', 'a', 'a', 'a', 'days', 'days', '.']
['SOS', 'i', 'm', 'untalented', 'charges', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'ready', 'we', 'can', 'make', 'safety', '.', 'EOS']
['SOS', 'i', 'am', 'the', 'for', 'for', 'for', 'the', 'job', '.']
['SOS', 'i', 'm', 'going', 'to', 'to', 'the', 'this', '.', 'EOS']
['SOS', 'you', 're', 'all', 'good', 'good', 'you', '?', 'EOS', 'PAD']
['SOS', 'you', 're', 'not', 'a', 'me', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'all', 'insane', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'at', 'the', 'the', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 'is', 'busy', 'very', 'strong', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'boring', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'used', 'to', 'the', 'alone', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'cured', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'not', 'as', 

['SOS', 'he', 's', 'very', 'upset', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'punctual', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'we', 're', 'not', 'married', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'a', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'tired', 'of', 'tired', 'in', 'now', '.', 'EOS']
['SOS', 'she', 's', 'my', 'my', 'age', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'she', 's', 'waiting', 'for', 'a', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'not', 'going', '.', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'you', 're', 'in', 'good', 'danger', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'we', 're', 'still', 'closer', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'glad', 'to', 'see', 'you', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'a', 'in', 'the', 'the', '.', '.', 'EOS']
['SOS', 'i', 'm', 'a', 'man', 'man', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'm', 'proud', 'of', 'my', 'children', '.', 'EOS', 'PAD']
['SOS', 'you', 're', 'so', 'mean', '.', 'EOS', 'PAD', 'PA

['SOS', 'i', 'm', 'very', 'to', 'to', 'your', 'your', 'homework', '.']
['SOS', 'we', 're', 'not', 'alone', 'busy', 'anymore', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'more', 'than', 'you', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'i', 'am', 'in', 'my', 'my', 'friends', '.', 'EOS', 'PAD']
['SOS', 'he', 's', 'a', 'journalist', '.', 'EOS', 'PAD', 'PAD', 'PAD']
['SOS', 'you', 're', 'very', 'very', 'productive', '.', '.', 'EOS', 'PAD']
['SOS', 'i', 'm', 'glad', 'to', 'see', 'you', 'here', '.', 'EOS']
['SOS', 'i', 'm', 'a', 'man', 'man', '.', 'EOS', 'PAD', 'PAD']
['SOS', 'he', 's', 'a', 'man', 'young', 'man', '.', 'EOS', 'PAD']
['SOS', 'she', 's', 'afraid', 'of', 'dogs', '.', '.', 'EOS', 'PAD']
['SOS', 'he', 'is', 'anxious', 'to', 'to', 'me', 'me', '.', 'EOS']
['SOS', 'you', 're', 'not', 'as', 'as', 'as', 'i', 'am', '.']
['SOS', 'she', 's', 'a', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'i', 'm', 'a', '.', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD']
['SOS', 'they', 're', 'all', 'me', '.', 'EOS', 'PAD', '

In [26]:
np.mean(getBlueScores(groundtruth_sentences[0:-53], output_sentences_beam))

/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/simar/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoothi

0.3089161572054788

In [27]:
output_sentences_beam

['SOS i m untalented . EOS',
 'SOS i m not as you . EOS',
 'SOS he s a student of . EOS',
 'SOS we we re all of of . EOS',
 'SOS she s teaching french french . EOS',
 'SOS she stood him . EOS',
 'SOS i m all now now now now . EOS',
 'SOS i m ready to to it . EOS',
 'SOS you re t aren t you ? EOS',
 'SOS you re so talented . EOS',
 'SOS we re all friends . EOS',
 'SOS i m trying for for a car . EOS',
 'SOS you re in of booze . EOS',
 'SOS we re getting for the wraps . EOS',
 'SOS we re just we we can can can .',
 'SOS i m really busy busy busy . EOS',
 'SOS i m as as as you . EOS',
 'SOS he s my my friend . EOS',
 'SOS you re both my friends friends . EOS',
 'SOS i m lazy . EOS',
 'SOS i m not a anymore . EOS',
 'SOS i m not sure sure . EOS',
 'SOS he is very likely to come . EOS',
 'SOS i m busy busy . EOS',
 'SOS i m glad to finally meet you . EOS',
 'SOS she s a a a hat . EOS',
 'SOS he s playing tv . EOS',
 'SOS i am not going to it . EOS',
 'SOS he s afraid of making mistakes . EOS